In [5]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
#Race Results

In [8]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [5]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP=GP[GP["Position"]!=0]
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,14,,,,Ferrari,,0,Luigi,Musso,Luigi Musso,0,0,2,0,10,NaT,NaT,NaT,0 days 03:07:03.400000,Finished,6,1957,British Grand Prix
1,10,,,,Ferrari,,0,Mike,Hawthorn,Mike Hawthorn,0,0,3,0,5,NaT,NaT,NaT,0 days 03:07:20.600000,Finished,4,1957,British Grand Prix
2,16,,,,Ferrari,,0,Peter,Collins,Peter Collins,0,0,4,0,9,NaT,NaT,NaT,NaT,+2 Laps,0,1957,British Grand Prix
3,16,,,,Ferrari,,0,Maurice,Trintignant,Maurice Trintignant,0,0,4,0,9,NaT,NaT,NaT,NaT,+2 Laps,3,1957,British Grand Prix
4,36,,,,Cooper,,0,Roy,Salvadori,Roy Salvadori,0,0,5,0,15,NaT,NaT,NaT,NaT,+5 Laps,2,1957,British Grand Prix


In [28]:
GP_Race_Wins=pd.pivot_table(GP[GP["Year"].isin(range(1957,2025))],index=['FullName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Race_Wins=GP_Race_Wins.fillna(0)
GP_Race_Wins.head()

count                                                         \
Position         1    2    3    4    5    6    7    8    9    10   11   12   
FullName                                                                     
Adrian Sutil    0.0  0.0  0.0  1.0  3.0  3.0  5.0  5.0  8.0  5.0  8.0  2.0   
Adrián Campos   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
Aguri Suzuki    0.0  0.0  1.0  0.0  0.0  4.0  3.0  4.0  2.0  3.0  2.0  3.0   
Al Herman       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Al Keller       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   

                                                                           \
Position         13   14   15    16    17    18    19   20   21   22   23   
FullName                                                                    
Adrian Sutil   13.0  6.0  7.0  10.0  11.0  10.0  12.0  6.0  7.0  5.0  1.0   
Adrián Campos   0.0  2.0  1.0   2.0   1.0   0.0   2.0  0.0  0.0  1.0  3.0   
Aguri Suzuki    2.0  2.0  1.0   5.0   2.0   4.0   4.0  2.0  4.0  5.0  4.0   
Al Herman       1.0  0.0  0.0   0.0   0.0   0.0   0.0  0.0  1.0  0.0  0.0   
Al Keller       0.0  0.0  0.0   0.0   0.0   1.0   0.0  0.0  0.0  0.0  0.0   

                                                  
Position        24   25   26   27   28   29   30  
FullName                                          
Adrian Sutil   0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Adrián Campos  1.0  4.0  0.0  2.0  0.0  1.0  0.0  
Aguri Suzuki   2.0  5.0  2.0  5.0  0.0  0.0  1.0  
Al Herman      0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Al Keller      0.0  0.0  0.0  1.0  0.0  0.0  0.0

In [22]:
#Remove the column that says full name
GP_Race_Wins.columns = GP_Race_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins=GP_Race_Wins.reset_index()

GP_Race_Wins=GP_Race_Wins.sort_values(by=["FullName"],ascending=[True])
GP_Race_Wins.head()

Position,FullName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,Adrian Sutil,0.0,0.0,0.0,1.0,3.0,3.0,5.0,5.0,8.0,5.0,8.0,2.0,13.0,6.0,7.0,10.0,11.0,10.0,12.0,6.0,7.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adrián Campos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,1.0,3.0,1.0,4.0,0.0,2.0,0.0,1.0,0.0
2,Aguri Suzuki,0.0,0.0,1.0,0.0,0.0,4.0,3.0,4.0,2.0,3.0,2.0,3.0,2.0,2.0,1.0,5.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,2.0,5.0,2.0,5.0,0.0,0.0,1.0
3,Al Herman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Al Keller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [23]:
import plotly.graph_objects as go

fig = go.Figure(go.Table
 (header={"values": GP_Race_Wins.columns},
  cells=dict(
    values=GP_Race_Wins.T.values,
    # fill_color=[GP_Race_Wins.Color],
    align='center', font=dict(color='white', size=18)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins.loc[GP_Race_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig.update_layout(
    title="Race Results",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig.show()

In [ ]:
#Sprint Results

In [24]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_sprint`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq3 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_10809/680041491.py:10: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [25]:
GP_sprint=df_bq3
GP_sprint=GP_sprint.fillna(0)
GP_sprint["Position"]=GP_sprint["Position"].astype(int)
GP_sprint["GridPosition"]=GP_sprint["GridPosition"].astype(int)
GP_sprint["Points"]=GP_sprint["Points"].astype(int)
GP_sprint=GP_sprint[GP_sprint["Position"]!=0]
GP_sprint.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,10,NaT,NaT,NaT,0 days 00:29:44.191000,Finished,0,2021,São Paulo Grand Prix
1,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,13,0,14,NaT,NaT,NaT,0 days 00:28:37.451000,Finished,0,2021,Italian Grand Prix
2,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,10,0,13,NaT,NaT,NaT,0 days 00:26:25.821000,Finished,0,2021,British Grand Prix
3,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,9,NaT,NaT,NaT,0 days 00:25:20.843000,Finished,0,2023,Belgian Grand Prix
4,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,7,0,8,NaT,NaT,NaT,0 days 00:31:03.332000,Finished,2,2023,Austrian Grand Prix


In [26]:
#Get the counts by final position in race

In [35]:
GP_Sprint_Wins=pd.pivot_table(GP_sprint[GP_sprint["Year"].isin(range(1957,2025))],index=['FullName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Sprint_Wins=GP_Sprint_Wins.fillna(0)
GP_Sprint_Wins.head()

count                                                    \
Position              1    2    3    4    5    6    7    8    9    10   11   
FullName                                                                     
Alexander Albon      0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  2.0  0.0  0.0   
Antonio Giovinazzi   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
Carlos Sainz         0.0  1.0  3.0  2.0  4.0  2.0  1.0  1.0  0.0  0.0  1.0   
Charles Leclerc      0.0  4.0  1.0  2.0  2.0  2.0  2.0  0.0  0.0  0.0  0.0   
Daniel Ricciardo     0.0  0.0  1.0  1.0  0.0  2.0  0.0  0.0  1.0  1.0  3.0   

                                                                 
Position             12   13   14   15   16   17   18   19   20  
FullName                                                         
Alexander Albon     1.0  2.0  0.0  1.0  1.0  2.0  1.0  0.0  1.0  
Antonio Giovinazzi  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  
Carlos Sainz        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Charles Leclerc     2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Daniel Ricciardo    2.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0

In [36]:
#Remove the column that says full name
GP_Sprint_Wins.columns = GP_Sprint_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins=GP_Sprint_Wins.reset_index()

GP_Sprint_Wins=GP_Sprint_Wins.sort_values(by=["FullName"],ascending=[True])
GP_Sprint_Wins.head()

Position,FullName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Alexander Albon,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0
1,Antonio Giovinazzi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Carlos Sainz,0.0,1.0,3.0,2.0,4.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Charles Leclerc,0.0,4.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Daniel Ricciardo,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,1.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
import plotly.graph_objects as go

fig_sprint = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins.columns},
  cells=dict(
    values=GP_Sprint_Wins.T.values,
    # fill_color=[GP_Sprint_Wins.Color],
    align='center', font=dict(color='white', size=14)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins.loc[GP_Sprint_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint.update_layout(
    title="Sprint Race Results",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig_sprint.show()

In [ ]:
#Team Race Results

In [ ]:
#Get the counts by final position in race

In [41]:
GP_Race_Wins_Team=pd.pivot_table(GP[GP["Year"].isin(range(1957,2025))],index=['TeamName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Race_Wins_Team=GP_Race_Wins_Team.fillna(0)
GP_Race_Wins_Team.head()

count                                                   \
Position             1    2    3    4    5    6     7     8     9     10   
TeamName                                                                   
AGS                 0.0  0.0  0.0  0.0  0.0  2.0   1.0   3.0   4.0   3.0   
ATS                 0.0  0.0  0.0  0.0  3.0  3.0   2.0   6.0   4.0   4.0   
Alfa Romeo          0.0  2.0  3.0  5.0  5.0  5.0   6.0  11.0  14.0  18.0   
Alfa Romeo Racing   0.0  0.0  0.0  1.0  1.0  0.0   3.0   4.0   7.0   8.0   
AlphaTauri          1.0  0.0  1.0  4.0  4.0  7.0  10.0   8.0  10.0  12.0   

                                                                               \
Position             11    12    13    14    15    16    17    18    19    20   
TeamName                                                                        
AGS                 1.0   3.0   4.0   1.0   2.0   4.0   2.0   1.0   3.0   2.0   
ATS                 9.0  10.0   9.0   7.0   7.0  10.0   4.0  12.0   3.0   4.0   
Alfa Romeo         16.0  22.0  19.0   8.0  18.0  21.0  18.0  15.0  15.0  15.0   
Alfa Romeo Racing  12.0  11.0  12.0  16.0  18.0   9.0   7.0   6.0   3.0   2.0   
AlphaTauri         17.0  14.0  11.0  13.0  14.0   5.0   7.0   7.0   7.0  14.0   

                                                                         
Position            21   22   23   24    25    26    27    28   29   30  
TeamName                                                                 
AGS                2.0  2.0  0.0  3.0   4.0   1.0   5.0  13.0  8.0  3.0  
ATS                3.0  6.0  3.0  6.0  12.0  10.0  10.0   8.0  5.0  2.0  
Alfa Romeo         9.0  4.0  8.0  8.0   9.0   4.0   1.0   0.0  0.0  0.0  
Alfa Romeo Racing  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  0.0  0.0  
AlphaTauri         0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0  0.0  0.0

In [42]:
#Remove the column that says full name
GP_Race_Wins_Team.columns = GP_Race_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins_Team=GP_Race_Wins_Team.reset_index()

GP_Race_Wins_Team=GP_Race_Wins_Team.sort_values(by=["TeamName"],ascending=[True])
GP_Race_Wins_Team.head()

Position,TeamName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,AGS,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,4.0,3.0,1.0,3.0,4.0,1.0,2.0,4.0,2.0,1.0,3.0,2.0,2.0,2.0,0.0,3.0,4.0,1.0,5.0,13.0,8.0,3.0
1,ATS,0.0,0.0,0.0,0.0,3.0,3.0,2.0,6.0,4.0,4.0,9.0,10.0,9.0,7.0,7.0,10.0,4.0,12.0,3.0,4.0,3.0,6.0,3.0,6.0,12.0,10.0,10.0,8.0,5.0,2.0
2,Alfa Romeo,0.0,2.0,3.0,5.0,5.0,5.0,6.0,11.0,14.0,18.0,16.0,22.0,19.0,8.0,18.0,21.0,18.0,15.0,15.0,15.0,9.0,4.0,8.0,8.0,9.0,4.0,1.0,0.0,0.0,0.0
3,Alfa Romeo Racing,0.0,0.0,0.0,1.0,1.0,0.0,3.0,4.0,7.0,8.0,12.0,11.0,12.0,16.0,18.0,9.0,7.0,6.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AlphaTauri,1.0,0.0,1.0,4.0,4.0,7.0,10.0,8.0,10.0,12.0,17.0,14.0,11.0,13.0,14.0,5.0,7.0,7.0,7.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
import plotly.graph_objects as go

fig_team = go.Figure(go.Table
 (header={"values": GP_Race_Wins_Team.columns},
  cells=dict(
    values=GP_Race_Wins_Team.T.values,
    # fill_color=[GP_Race_Wins_Team.Color],
    align='center', font=dict(color='white', size=12)
    ))
 )

# fig_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins_Team.loc[GP_Race_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_team.update_layout(
    title="Race Results By Team",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig_team.show()

In [ ]:
#Team Sprint Race Results

In [44]:
GP_Sprint_Wins_Team=pd.pivot_table(GP_sprint[GP_sprint["Year"].isin(range(1957,2025))],index=['TeamName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.fillna(0)
GP_Sprint_Wins_Team.head()

count                                                    \
Position             1    2    3    4    5    6    7    8    9    10   11   
TeamName                                                                    
Alfa Romeo          0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  2.0  0.0   
Alfa Romeo Racing   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
AlphaTauri          0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  2.0  1.0   
Alpine              0.0  0.0  1.0  0.0  0.0  1.0  3.0  0.0  5.0  1.0  3.0   
Aston Martin        0.0  0.0  0.0  1.0  1.0  1.0  0.0  3.0  1.0  3.0  2.0   

                                                                
Position            12   13   14   15   16   17   18   19   20  
TeamName                                                        
Alfa Romeo         1.0  2.0  2.0  2.0  2.0  2.0  0.0  2.0  2.0  
Alfa Romeo Racing  0.0  2.0  0.0  1.0  0.0  0.0  2.0  0.0  0.0  
AlphaTauri         3.0  0.0  2.0  3.0  3.0  3.0  1.0  1.0  2.0  
Alpine             2.0  4.0  1.0  3.0  1.0  2.0  2.0  0.0  1.0  
Aston Martin       2.0  4.0  3.0  2.0  1.0  1.0  0.0  2.0  3.0

In [45]:
#Remove the column that says full name
GP_Sprint_Wins_Team.columns = GP_Sprint_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.reset_index()

GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.sort_values(by=["TeamName"],ascending=[True])
GP_Sprint_Wins_Team.head()

Position,TeamName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Alfa Romeo,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0
1,Alfa Romeo Racing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0
2,AlphaTauri,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,1.0,3.0,0.0,2.0,3.0,3.0,3.0,1.0,1.0,2.0
3,Alpine,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,1.0,3.0,2.0,4.0,1.0,3.0,1.0,2.0,2.0,0.0,1.0
4,Aston Martin,0.0,0.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,3.0,2.0,2.0,4.0,3.0,2.0,1.0,1.0,0.0,2.0,3.0


In [48]:
import plotly.graph_objects as go

fig_sprint_team = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins_Team.columns},
  cells=dict(
    values=GP_Sprint_Wins_Team.T.values,
    # fill_color=[GP_Sprint_Wins_Team.Color],
    align='center', font=dict(color='white', size=20)
    ))
 )

# fig_sprint_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins_Team.loc[GP_Sprint_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint_team.update_layout(
    title="Sprint Race Results By Team",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=20,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig_sprint_team.show()